<a href="https://colab.research.google.com/github/Nubiancodingdelight/ACS-Project-Repository-/blob/main/Generating_VIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mounts google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#set path
%cd /content/drive/MyDrive/ACS_Research/VISDB_Data/

/content/drive/MyDrive/ACS_Research/VISDB_Data


In [2]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

Extract sequence from virus

In [ ]:
def virus_seq_extract(ref, start, stop):

  return seq

Extract Sequence from Human Reference 100 bp upstream and 100 bp downstream

In [ ]:
def human_seq_extract(ref, start, stop):

  return seq_list

Load Data

In [5]:
#load xslx
vis_df = pd.read_excel('Supplementary Table 1.xlsx')
vis_df.head()

,Virus,Virus subtype,Reference virus genome,Begin breakpoint,Stop breakpoint,Chromosome,Locus,Reference human genome,Begin location,End location,Sample,Disease,Method,PubMed ID,Target gene
0,HBV,NaN,FJ562281.1,1782.0,NaN,chr9,q31.1,GRCh38,100940295.0,NaN,Tumor,Hepatocellular carcinoma,PCR and Sanger sequencing,30535432,CORO2A
1,HBV,NaN,GQ377546.1,1820.0,NaN,chr9,q31.1,GRCh38,101030026.0,NaN,Tumor,Hepatocellular carcinoma,PCR and Sanger sequencing,30535432,NaN
2,HBV,NaN,KC792890.1,1471.0,NaN,chr3,p26.3,GRCh38,2002381.0,NaN,Tumor,Hepatocellular carcinoma,PCR and Sanger sequencing,30535432,NaN
3,HBV,NaN,FJ622847.1,689.0,NaN,chr2,q35,GRCh38,216281034.0,NaN,Tumor,Hepatocellular carcinoma,PCR and Sanger sequencing,30535432,NaN
4,HBV,NaN,KF166416.1,1807.0,NaN,chr16,q12.1,GRCh38,51320015.0,NaN,Tumor,Hepatocellular carcinoma,PCR and Sanger sequencing,30535432,NaN


Transform Data

In [ ]:
def lower_case_and_underscore(df):
  """Lowercases all column names and replaces spaces with underscores."""
  new_columns = [col.lower().replace(' ', '_') for col in df.columns]
  df.columns = new_columns
  return df

In [6]:
vis_df = lower_case_and_underscore(vis_df)



,virus,virus_subtype,reference_virus_genome,begin_breakpoint,stop_breakpoint,chromosome,locus,reference_human_genome,begin_location,end_location,sample,disease,method,pubmed_id,target_gene
0,HBV,NaN,FJ562281.1,1782.0,NaN,chr9,q31.1,GRCh38,100940295.0,NaN,Tumor,Hepatocellular carcinoma,PCR and Sanger sequencing,30535432,CORO2A
1,HBV,NaN,GQ377546.1,1820.0,NaN,chr9,q31.1,GRCh38,101030026.0,NaN,Tumor,Hepatocellular carcinoma,PCR and Sanger sequencing,30535432,NaN
2,HBV,NaN,KC792890.1,1471.0,NaN,chr3,p26.3,GRCh38,2002381.0,NaN,Tumor,Hepatocellular carcinoma,PCR and Sanger sequencing,30535432,NaN
3,HBV,NaN,FJ622847.1,689.0,NaN,chr2,q35,GRCh38,216281034.0,NaN,Tumor,Hepatocellular carcinoma,PCR and Sanger sequencing,30535432,NaN
4,HBV,NaN,KF166416.1,1807.0,NaN,chr16,q12.1,GRCh38,51320015.0,NaN,Tumor,Hepatocellular carcinoma,PCR and Sanger sequencing,30535432,NaN


In [13]:
# prompt: filter out rows where begin_breakpoint or stop breakpoint  and also begin and end location are NA

vis_df = vis_df.dropna(subset=['begin_breakpoint', 'stop_breakpoint', 'begin_location', 'end_location', 'reference_virus_genome'], how='any')
vis_df.head()

,virus,virus_subtype,reference_virus_genome,begin_breakpoint,stop_breakpoint,chromosome,locus,reference_human_genome,begin_location,end_location,sample,disease,method,pubmed_id,target_gene
346,HBV,NaN,X70185.1,1720.0,1603.0,chr1,q21.1,GRCh37/hg19,420.0,437.0,cell,Hepatocellular carcinoma,invPCR+NAH,28414318,ANP32E
347,HBV,NaN,X70185.1,1246.0,1390.0,chr1,p21.3,GRCh37/hg19,97732036.0,97732056.0,cell,Hepatocellular carcinoma,invPCR+NAH,28414318,DPYD
348,HBV,NaN,X70185.1,1603.0,1664.0,chr1,q31.3,GRCh37/hg19,194161891.0,194161910.0,cell,Hepatocellular carcinoma,invPCR+NAH,28414318,NaN
349,HBV,NaN,X70185.1,1623.0,1717.0,chr2,q31.1,GRCh37/hg19,171450967.0,171450816.0,cell,Hepatocellular carcinoma,invPCR+NAH,28414318,MYO3B
350,HBV,NaN,X70185.1,1638.0,1717.0,chr2,q31.1,GRCh37/hg19,171450967.0,171450816.0,cell,Hepatocellular carcinoma,invPCR+NAH,28414318,MYO3B


In [14]:
print('Number of Rows: ' + str(len(vis_df)))
vis_df['reference_virus_genome'].unique()

Number of Rows: 1509


array(['X70185.1', 'AB033556.1', 'X75657.1', 'NC_003977.1', 'NC_003977',
       'AY033073.1', 'V01460.1', 'X0276', 'V00866', 'AF068756 ',
       'AF330110'], dtype=object)

Concatenate Viral and Human Sequence and Insert into

In [ ]:
#loop through data
seq_list =[]
for row,ind in vis_df:
  virus_name = row['virus']
  virus_ref = row['reference_virus_genome']
  hum_ref = row['reference_human_genome']
  human_seq = human_seq_extract(hum_ref, row['begin_location'],row['start_location'])
  viral_seq = virus_seq_extract(virus_ref, row['begin_breakpoint'], row['stop_breakpoint'])
  spliced_seq = human_seq[0] + viral_seq + human_seq[1]
  seq_list.append([virus_name, virus_ref, hum_ref, spliced_seq, viral_seq, human_seq[0], human_seq[1]])



In [ ]:
#make seq_list a dataframe save as a csv
seq_df = pd.DataFrame(seq_list, columns=['virus', 'virus_ref', 'human_ref', 'spliced_seq', 'viral_seq', 'human_seq_upstream', 'human_seq_downstream'])
